# Setup

In [1]:
import numpy as np
import math

Need to use `.item()` as per https://stackoverflow.com/questions/24565916/why-is-numpy-shape-empty (and also Frank's message)

Also note that `mat1` contains only protons, and `mat2` contains only iron nuclei.

In [2]:
mat1 = np.load('../data/sim_12360_00.npy').item()
mat2 = np.load('../data/sim_12362_00.npy').item()

# Exploration

In [3]:
mat1.keys()

dict_keys(['Charges', 'Energy', 'File_info', 'dir_reco', 'core_MC', 'Gain', 'core_reco', 'Position', 'dir_MC', 'Fit_status', 'Composition'])

Each index in each array corresponds to one cosmic ray.
* Charges: dictionaries, where each dictionary has about (but not exactly!) 6-8 values. Each key is a sensor and each value is the value that sensor gathered. Presumably the sensors that did not recieve significant signal are not included.
* Energy: one value per event. Unclear what that value is.
* File_info: some information about the file that the event was drawn from 
* Gain: a list with one item-- a dictionary from sensor to whether it is a high-gain or low-gain sensor. 
* Position: the position of each sensor. Consists of an x-position array and a y-position array, but it is unclear what order they are in. 
* Fit_status: mostly `'OK'` with some `'InsufficientHits'`. Unclear where that comes into play.
* Composition: a string representing the composition of each ray.

In [ ]:
len(mat1['dir_MC'])

# Clean up dir_MC and core_MC

In [4]:
def cleanup(li):
    new_list = []
    for i in range(len(li)//2):
        new_list.append((li[2*i], li[2*i+1]))
    return new_list

In [5]:
matrices = (mat1, mat2)
keys = ('dir_MC', 'core_MC', 'dir_reco', 'core_reco')
for matrix in matrices:
    for key in keys:
        matrix[key] = cleanup(matrix[key])

In [6]:
len(mat1['dir_reco'])

16531

## Questions

* Do the x/y positions correlate to some real-world data? Is it possible to, for example, normalize to all positive?
 * `d['Position']` is given in meters from the center of the detector. You can translate them so long as you know you're moving the detector center.
* What does `mat1['File_info']` contain? My instinct is some information about the original simulation files that this was drawn from but perhaps not.
 * This is correct.
* Is it the case that sensors that did not see any signal are not included in `'Charges'`?
 * `d['Charges']` does only contain triggered DOMs. This was actually a major motivation for the dictionary format, so we don't have to store all those zeros.
* What is stored under `Energy`?
 * `d['Energy']` represents the true energy of the initial cosmic-ray that caused the shower. It's given in giga-electronvolts (GeV), but we typically look at it on a log-scale, so the simulation runs from 5-8 in log10(E/GeV).
* Is the `position` array `[x_dict, y_dict]` or `[y_dict, x_dict]`?
 * Frank didn't explicitly answer this one but it seems like x,y.
* Do we have access to the actual/calculated initial conditions information (angle/center) or is that something we need to calculate? If the latter, is there a script somewhere?
 * `d['Energy']` is one of those quantities. We can also pull initial direction, a variety of reconstructed directions, and potentially a reconstructed energy value.


# Build a gain-differentiated dataset

Find all the sensors that are high/low gain

In [7]:
hgain = list(filter(lambda x: mat1['Gain'][0][x] == 'High', mat1['Position'][0].keys()))
lgain = list(filter(lambda x: mat1['Gain'][0][x] == 'Low', mat1['Position'][0].keys()))
all_sensors = mat1['Gain'][0].keys()

Construct a dictionary that goes from name -> column index in the matrix we're about to build

In [8]:
#build a name->index dict
def get_index_dict(sensors, direction_data=False):
    name_index_dict = {}
    for i in range(len(hgain)):
        name_index_dict[hgain[i]] = i
    if direction_data:
        name_index_dict['zenith'] = i+1
        name_index_dict['azimuth'] = i+2
    return name_index_dict

In [9]:
hgain_indices = get_index_dict(hgain)
lgain_indices = get_index_dict(lgain)
all_indices = get_index_dict(all_sensors)

Build the matrix by setting up an empty one and populating it with the values according to the indices we created above. This is pretty slow atm but I'm sure there are ways to make it more efficient. Perhaps we don't need to make it dense in order to pop it into numpy? 

In [10]:
def gen_matrix(proton_data, iron_data, feature_dict, direction=False):
    event_matrix = np.zeros((len(proton_data['Composition']) + len(iron_data['Composition']), len(feature_dict) + 1))
    for i in range(len(proton_data['Charges'])):
        #insert sensor data
        event = proton_data['Charges'][i]
        for sensor in proton_data['Charges'][i].keys():
            try:
                event_matrix[i, feature_dict[sensor]] = 0 if math.isnan(event[sensor]) else event[sensor]
            except KeyError:
                continue
        #insert direction data
        if direction:
            event_matrix[i, feature_dict['zenith']] = proton_data['dir_MC'][i][0]
            event_matrix[i, feature_dict['azimuth']] = proton_data['dir_MC'][i][1]
        event_matrix[i, -1] = 1
    
    for i in range(len(iron_data['Charges'])):
        #insert sensor data
        event = iron_data['Charges'][i]
        for sensor in iron_data['Charges'][i].keys():
            try:
                event_matrix[len(proton_data['Charges']) + i, feature_dict[sensor]] = 0 if math.isnan(event[sensor]) else event[sensor]
            except KeyError:
                continue
        
        #insert direction data
        if direction:
            event_matrix[i, feature_dict['zenith']] = iron_data['dir_MC'][i][0]
            event_matrix[i, feature_dict['azimuth']] = iron_data['dir_MC'][i][1]    
        event_matrix[len(proton_data['Charges']) + i, -1] = 0
        
    return event_matrix

In [11]:
hgain_events = gen_matrix(mat1, mat2, hgain_indices)
lgain_events = gen_matrix(mat1, mat2, lgain_indices)

In [12]:
hgain_events.shape

(31620, 163)

# Split high-gain into training/test set

In [ ]:
np.random.shuffle(hgain_events)
train_size = int(hgain_events.shape[0] * .9)

trainset = hgain_events[:train_size]
testset = hgain_events[train_size:]

In [ ]:
trainset[trainset[:,-1]==0].shape

In [ ]:
np.save('small_train.npy', trainset)
np.save('small_test.npy', testset)

# Train/Test set with all the data

In [ ]:
all_events = gen_matrix(mat1, mat2, all_indices)
np.random.shuffle(all_events)
train_size = int(all_events.shape[0] * .9)

trainset = all_events[:train_size]
testset = all_events[train_size:]

In [ ]:
trainset[trainset[:,-1]==0].shape

In [ ]:
np.save('small_train_high_low.npy', trainset)
np.save('small_test_high_low.npy', testset)

# Add azimuth, zenith features to dataset

In [13]:
all_indices_dir = get_index_dict(all_sensors, direction_data=True)

In [14]:
all_events_direction = gen_matrix(mat1, mat2, all_indices_dir, direction=True)
np.random.shuffle(all_events_direction)
train_size = int(all_events_direction.shape[0] * .9)

trainset = all_events_direction[:train_size]
testset = all_events_direction[train_size:]

In [15]:
all_events_direction.shape

(31620, 165)

In [16]:
np.save('direction_train.npy',trainset)
np.save('direction_test.npy', testset)

In [ ]:
len(mat1['Charges']) + len(mat2['Charges'])

# Exploring width of charge readings
In this section, I'm trying to figure out how widely the charge readings go. How easy will it be to trim the number of input features to those around the centroid?

In [ ]:
def get_extreme_pos(charge_dict, positions, big=True):
    l = list(map(lambda x:positions[x], charge_dict.keys()))
    if big:
        return max(l)
    return min(l)

def get_bounding_box(charge_dict, positions):
    min_x = get_extreme_pos(charge_dict, positions[0], big=False)
    max_x = get_extreme_pos(charge_dict, positions[0], big=True)
    min_y = get_extreme_pos(charge_dict, positions[1], big=False)
    max_y = get_extreme_pos(charge_dict, positions[1], big=True)
    return (min_x, max_x, min_y, max_y)

def get_dimensions(charge_dict, positions):
    min_x, max_x, min_y, max_y = get_bounding_box(charge_dict, positions)
    return max_x - min_x, max_y - min_y

dimensions = [get_dimensions(event, mat1['Position']) for event in mat1['Charges']]

widest = max(dimensions, key=lambda x: x[0])
tallest = max(dimensions, key=lambda x: x[1])
print(widest, tallest)
# width, height = get_dimensions(mat1['Charges'][0], mat1['Position'])
# print('width: {} height: {}'.format(width, height))
# print('X: ({},{}), Y: ({},{})'.format(min_x,max_x,min_y,max_y))

In [ ]:
width = max(mat1['Position'][0].values()) - min(mat1['Position'][0].values())
height = max(mat1['Position'][1].values()) - min(mat1['Position'][1].values())
print('{} {}'.format(width, height))

So, we can't willy-nilly trim to the max width, since at least one reading spreads across the entire height or width. 

The next thing to try is some kind of center-of-mass trimming. First, I want to find out the average width and height so we can see how much trimming seems appropriate.

In [ ]:
widths = [x[0] for x in dimensions]
heights = [x[1] for x in dimensions]
print('width: {} height: {}'.format(np.median(widths), np.median(heights)))

In [ ]:
def get_com(charge_dict, positions):
    xs = [positions[0][x] for x in charge_dict.keys()]
    ys = [positions[1][y] for y in charge_dict.keys()]
    weights = [x for x in charge_dict.values()]
    x = np.average(xs, weights=weights)
    y = np.average(ys, weights=weights)
    return(x,y)

def get_positions(positions_dict_x, positions_dict_y):
    positions = {}
    for sensor in positions_dict_x.keys():
        positions[sensor] = (positions_dict_x[sensor], positions_dict_y[sensor])
    return positions

def get_coms(charge_dicts, positions):
    return [get_com(event, positions) for event in charge_dicts]

def in_bounds(position, x_bound, y_bound):
    in_bounds_x = position[0] <= x_bound[1] and position[0] >= x_bound[0]
    in_bounds_y = position[1] <= y_bound[1] and position[1] >= y_bound[0]
    return in_bounds_x and in_bounds_y

def get_trimmed_charge_dict(charge_dict, positions, x_bound, y_bound):
    return {key : value for key,value in charge_dict.items() if in_bounds((positions[0][key], positions[1][key]), x_bound, y_bound)}

def get_trimmed_charge_dicts(charge_dicts, positions):
    coms = get_coms(charge_dicts, positions)
    normalized_positions = {}
    

In [ ]:
get_bounding_box(mat1['Charges'][0], mat1['Position'])

In [ ]:
trimmed = get_trimmed_charge_dict(mat1['Charges'][0], mat1['Position'], (0, 160), (0, 76))
# list(map(lambda x: (mat1['Position'][0][x],mat1['Position'][1][x]), trimmed.keys()))
trimmed

In [ ]:
positions = get_positions(mat1['Position'][0], mat1['Position'][1])
positions

In [ ]:
targs = get_coms(mat1['Charges'],mat1['Position'])

#pos_a: 
def get_distance(pos_a, pos_b):
    return np.sqrt(np.sum(np.power(np.array(pos_a)-np.array(pos_b), 2)))

def get_distances(target, all_positions):
    return {sensor: get_distance(target, all_positions[sensor]) for sensor in all_positions.keys()}

def get_nearest_neighbor(target, all_positions):
    distances = {sensor: get_distance(target, all_positions[sensor]) for sensor in all_positions.keys()}
    min_sensor = 0
    min_value = 99999999999999999999
    for sensor in distances.keys():
        if distances[sensor] < min_value:
            min_sensor = sensor
            min_value = distances[sensor]
    return min_sensor

# d = get_distances(targ, positions)
# min_sensor = 0
# min_value = 999999999999999999
# for sensor in d.keys():
#     if d[sensor] < min_value:
#         min_sensor = sensor
#         min_value = d[sensor]
# min_sensor
res = []
for center in targs:
    res.append(get_nearest_neighbor(center, positions))
    if(len(res) % 500 == 0):
        print(len(res) / len(targs))